In [1]:
!pip install -Uqqq pip --progress-bar off
print("Installed PIP")
!pip install -Uqqq git+https://github.com/huggingface/transformers
print("Installed Transformers")
!pip install -qqq -U peft
print("Installed PEFT")
!pip install -Uqqq accelerate
print("Installed Accelerate")
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
print("Installed LoRA Lib")
!pip install -qqq bitsandbytes
print("Installed BitsAndBytes")
!pip install fsspec -qqq

!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0 -qqq
print("Installed Datasets")

Installed PIP
Installed Transformers
Installed PEFT
Installed Accelerate
Installed LoRA Lib
Installed BitsAndBytes
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.4.0 which 

In [2]:
!pip install -Uqqq torch torchvision
print("Installed Torch")

Installed Torch


In [3]:
!pip install huggingface_hub -qqq

In [4]:
from kaggle_secrets import UserSecretsClient

import wandb

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("text-summarizer")
secret_value_1 = user_secrets.get_secret("wandb")

In [5]:
import huggingface_hub

wandb.login(key = secret_value_1)

huggingface_hub.login(token=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig  
)

In [7]:
# MODEL_NAME = "akoksal/LongForm-OPT-125M"

MODEL_NAME = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='auto',
    quantization_config=bnb_config,
    use_cache=False,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        
        if param.requires_grad:
            trainable_params += param.numel()
    
    print(
        f"Trainable params: {trainable_params} || All params: {all_param} || trainable %: {100* trainable_params/all_param}"
    )

In [9]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
#     target_modules=['query_key_value'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

Trainable params: 18350080 || All params: 1539742720 || trainable %: 1.1917627381280946


In [11]:
gen_config = model.generation_config
gen_config.max_new_tokens = 200
gen_config.temperature = 0.7
gen_config.top_p = 0.7
gen_config.num_return_sequences = 1
gen_config.pad_token_id = tokenizer.eos_token_id
gen_config.eos_token_id = tokenizer.eos_token_id

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

filename = '/kaggle/input/ind-proj/paths.csv'

df = pd.read_csv(filename, encoding="utf-8", encoding_errors="replace")

df = df[['summary', 'paths']]

train, test  = train_test_split(df, 
                                train_size=.8,
                                test_size=.2, 
                                random_state=42)

In [13]:
from datasets import DatasetDict, Dataset

train_ds = Dataset.from_pandas(train).remove_columns(["__index_level_0__"])

eval_ds = Dataset.from_pandas(test).remove_columns(["__index_level_0__"])

paths_ds = DatasetDict(
{
    'train': train_ds,
    'test': eval_ds
})

In [14]:
def generate_prompt(data_point):
    
    """"
    Update the prompt template:
    Combine both the prompt and input into a single column.

    """     
    bos_token = "<s>"
    
    original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    
    system_message = "Use the provided context followed by a question to answer it."
    
    full_prompt = f"""Act as the author of a Choose Your Own Adventure Book. This book is special as it is based on existing material.
    Now, as with any choose your own adventure book, you'll have to generate decision paths at certain points in the story.
    Your job is to generate 4 decision paths for the given point in the story, if applicable to that point in the story.
    If the given part of the story doesn't contain any decisions from which to generate decision paths, don't
    generate any. If the given part of the story contains a decision, generate 4 decision paths for that decision.
    One among the 4 decision paths should be the original path, the other 3 should deviate from the original path in a sensible manner.
    The decision paths should be generated in a way that they are coherent with the existing story.
    The result should be a JSON object with the following keys: [text, paths]

    text: The given text
    paths: The generated decision paths as strings in a list
    
    Text:
    {data_point['summary']}
    
    Paths:
    {eval(data_point['paths'])}
    """
    
    full_prompt = " ".join(full_prompt.split())
    
    return full_prompt

In [15]:
def generate_and_tokenize_prompt(data_point):
    
    full_prompt = generate_prompt(data_point)
    
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    
    return tokenized_full_prompt

In [16]:
train_dataset = paths_ds["train"].shuffle().map(generate_and_tokenize_prompt)
val_dataset = paths_ds["test"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/659 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

In [17]:
OUTPUT_DIR = "mistral-gen"

In [18]:
train_dataset

Dataset({
    features: ['summary', 'paths', 'input_ids', 'attention_mask'],
    num_rows: 659
})

In [19]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
#     num_train_epochs = 1,
    learning_rate = 2e-4,
    fp16 = True,
    save_total_limit = 3,
    logging_steps = 10,
    output_dir = OUTPUT_DIR,
    max_steps = 250,
    optim = "paged_adamw_8bit",
    lr_scheduler_type = "cosine",
    warmup_ratio = 0.05,
)

2024-04-22 23:15:45.225062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 23:15:45.225178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 23:15:45.479805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [20]:
trainer = transformers.Trainer(
    model = model,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    args = training_args,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

max_steps is given, it will override any value given in num_train_epochs


In [21]:
model.config_use_cache = False

trainer.train()

wandb: Currently logged in as: n-suneel89 (n-suneel-duke). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.593600
20,1.750700
30,1.211700
40,1.324400
50,1.215500
60,1.137700
70,1.149100
80,1.217200
90,1.088200
100,1.140500


TrainOutput(global_step=250, training_loss=1.185545139312744, metrics={'train_runtime': 1467.7821, 'train_samples_per_second': 0.681, 'train_steps_per_second': 0.17, 'total_flos': 5260550383656960.0, 'train_loss': 1.185545139312744, 'epoch': 1.5174506828528074})

In [22]:
trainer.push_to_hub()

events.out.tfevents.1713827759.08d1a6d43a49.34.0:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/73.4M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/suneeln-duke/mistral-gen/commit/20eb322866997494a9e4284eb59fd890e3ad9cda', commit_message='End of training', commit_description='', oid='20eb322866997494a9e4284eb59fd890e3ad9cda', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
def generate_test_prompt(summary):
    
    """"
    Update the prompt template:
    Combine both the prompt and input into a single column.

    """     
    
    full_prompt = f"""Act as the author of a Choose Your Own Adventure Book. This book is special as it is based on existing material.
    Now, as with any choose your own adventure book, you'll have to generate decision paths at certain points in the story.
    Your job is to generate 4 decision paths for the given point in the story, if applicable to that point in the story.
    If the given part of the story doesn't contain any decisions from which to generate decision paths, don't
    generate any. If the given part of the story contains a decision, generate 4 decision paths for that decision.
    One among the 4 decision paths should be the original path, the other 3 should deviate from the original path in a sensible manner.
    The decision paths should be generated in a way that they are coherent with the existing story.
    The result should be a JSON object with the following keys: [text, paths]

    text: The given text
    paths: The generated decision paths as strings in a list
    
    Text:
    {summary}
    
    Paths:
    """
    
    full_prompt = " ".join(full_prompt.split())
    
    return full_prompt

In [24]:
from transformers import pipeline

OUTPUT_DIR = "mistral-gen"

#  = pipeline( )


path_gen = pipeline(task="text-generation", 
                        model=f"suneeln-duke/{OUTPUT_DIR}", 
                        tokenizer=tokenizer,
                        max_new_tokens = 512, 
                        temperature = 0.0,
                       )

adapter_config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
paths_ds['test'][0]

{'summary': 'Lord Bholenath, where will he be in the three worlds? Bhole! Mr. Banwari, have they come? Yes, l know. You dance the entire night, flirt around.. and sleep peacefully during the day . You have got a habit of forgetting. What will we do till they come, sir? You wear a skirt. l will wear pants. We will dance in front of them . Okay, Sir, juice? You drink it. Drink it. Hi, uncle',
 'paths': "['Continue dancing and flirting at the party', 'Leave the party early and go home', 'Start a deep conversation with Mr. DP about his house', 'Sneak out of the party to meet GK who is out of lndia']"}

In [26]:
sample = generate_test_prompt(paths_ds['test'][0]['summary'])

In [27]:
sample

"Act as the author of a Choose Your Own Adventure Book. This book is special as it is based on existing material. Now, as with any choose your own adventure book, you'll have to generate decision paths at certain points in the story. Your job is to generate 4 decision paths for the given point in the story, if applicable to that point in the story. If the given part of the story doesn't contain any decisions from which to generate decision paths, don't generate any. If the given part of the story contains a decision, generate 4 decision paths for that decision. One among the 4 decision paths should be the original path, the other 3 should deviate from the original path in a sensible manner. The decision paths should be generated in a way that they are coherent with the existing story. The result should be a JSON object with the following keys: [text, paths] text: The given text paths: The generated decision paths as strings in a list Text: Lord Bholenath, where will he be in the three 

In [28]:
%%time

result = path_gen(sample, pad_token_id=path_gen.tokenizer.eos_token_id)
answer = result[0]['generated_text'].split("=")[-1].lower()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:500: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


CPU times: user 47.4 s, sys: 802 ms, total: 48.2 s
Wall time: 24.1 s


In [29]:
answer

"act as the author of a choose your own adventure book. this book is special as it is based on existing material. now, as with any choose your own adventure book, you'll have to generate decision paths at certain points in the story. your job is to generate 4 decision paths for the given point in the story, if applicable to that point in the story. if the given part of the story doesn't contain any decisions from which to generate decision paths, don't generate any. if the given part of the story contains a decision, generate 4 decision paths for that decision. one among the 4 decision paths should be the original path, the other 3 should deviate from the original path in a sensible manner. the decision paths should be generated in a way that they are coherent with the existing story. the result should be a json object with the following keys: [text, paths] text: the given text paths: the generated decision paths as strings in a list text: lord bholenath, where will he be in the three 